# Topic Modeling Using LDA

In [ ]:
# We

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import re
import numpy as np
import pandas as pd
import nltk
import sklearn
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as plt_gensim # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping


In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english') # add more
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
stop_words.extend(['food','din','tai','fung','cuban','restaurant'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ysabelgonzalezrico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
reviews = pd.read_csv('/Users/ysabelgonzalezrico/Documents/UCI/4 - SPRING/NLP/Yelp/reviewsfinal.csv')
reviews

,username,location,date,stars,content,numfriends,restaurantname
0,Sarah E.,"Arcadia, CA",4/2/22,5,Been going here for 5+ years starting at the o...,247,Din Tai Fung
1,Cynthia W.,"Irvine, CA",3/17/22,4,Very professional and organized. Wonderful sta...,0,Din Tai Fung
2,Julienne C.,"Buena Park, CA",4/24/22,5,We came here for our anniversary as walk ins a...,0,Din Tai Fung
3,Tina N.,"Atlanta, GA",5/1/22,4,DTF? Obviously had to go to DTF for the one ti...,2,Din Tai Fung
4,Dr. Susan F.,"Whittier, CA",4/27/22,1,We love DTF!!! Not this location: is there an...,16,Din Tai Fung
...,...,...,...,...,...,...,...
20980,Rick P.,"Long Beach, CA",7/5/06,3,i agree with eugenia c. when she says the serv...,198,Habana
20981,Shannon W.,"Costa Mesa, CA",2/24/13,1,"Ok, I understand why this place gets the revie...",0,Habana
20982,Thenameisfood T.,"Hacienda Heights, CA",9/11/12,4,pretty good food! bit pricey.. the Cuban Food ...,514,Habana
20983,Brett M.,"Huntington Beach, CA",3/14/10,5,"very cool dark romantic restaurant, great food...",1,Habana


In [5]:
reviews1 = reviews.dropna(how='any',axis=0)

# Negative sentiment topics

In [28]:
# Create negative reviews dataset (1 and 2 star reviews )

neg_sent = reviews1.loc[((reviews1.stars==1) | (reviews1.stars==2))]
neg_sent

,username,location,date,stars,content,numfriends,restaurantname
4,Dr. Susan F.,"Whittier, CA",4/27/22,1,We love DTF!!! Not this location: is there an...,16,Din Tai Fung
36,Nba C.,"Irvine, CA",4/24/22,1,You people are fools. Paying for mediocre Chin...,10,Din Tai Fung
40,Tiffany O.,"Orange County, CA",4/30/22,1,Ordered and was told it would be ready at 7:30...,0,Din Tai Fung
48,Rocky H.,"Orange, CA",3/26/22,2,"Our family is pretty big, 7 adults and 4 kids....",0,Din Tai Fung
54,Mike L.,"Irvine, CA",4/6/22,1,We had a reservation at 11 am on Saturday. We ...,0,Din Tai Fung
...,...,...,...,...,...,...,...
20966,Prince Of O.,"Huntington Beach, CA",1/25/12,1,the atmosphere was good and service ok.the foo...,0,Habana
20974,Alonso C.,"Santa Ana, CA",1/4/11,1,My wife and i always go shop at urbanoutfitter...,0,Habana
20978,Kayla N.,"San Mateo, CA",7/5/13,2,Went here for Mother's day expecting a great e...,0,Habana
20979,Kory C.,"Long Beach, CA",8/18/11,1,"So, I didn't even eat here but i work at the W...",6,Habana


In [29]:
df = neg_sent[['content','stars']]
df

,content,stars
4,We love DTF!!! Not this location: is there an...,1
36,You people are fools. Paying for mediocre Chin...,1
40,Ordered and was told it would be ready at 7:30...,1
48,"Our family is pretty big, 7 adults and 4 kids....",2
54,We had a reservation at 11 am on Saturday. We ...,1
...,...,...
20966,the atmosphere was good and service ok.the foo...,1
20974,My wife and i always go shop at urbanoutfitter...,1
20978,Went here for Mother's day expecting a great e...,2
20979,"So, I didn't even eat here but i work at the W...",1


In [30]:
# Convert to list
data = df.content.values.tolist()

#remove quotes
data = [re.sub('\"', ' ', review) for review in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove single quotes
data = [re.sub("\'", " ", sent) for sent in data]

pprint(data[:1])


['We love DTF!!! Not this location: is there anything good we can say about '
 'this location?? NO!!! From the moment we walked up to put our party of two '
 'name in at 3:20 p.m. on a Monday afternoon, it was awful!!! The two '
 'receptionists talked in whispers behind their masks and when we asked them '
 'to repeat what they were saying, the spoke at 100 mph. shouting! We are not '
 'deaf nor stupid and we should have walked away right then. We did not.Our '
 'order was simple: a veggie, two bun orders, appetizer, and noodles plus hot '
 'tea. The meal took a total of 70 minutes to be served as it dripped and '
 'drabbled out of the kitchen; the food was cold, tough, unappetizing. The '
 'wait staff was snotty, arrogant, and standing around on a noisy floor with '
 'open tables everywhere. Our server kept apologizing saying, the kitchen has '
 'some problems today; I am trying to rush each segment out. Would you like '
 'more tea? Our bill was just a few dollars short of $100 for a 

In [31]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['we', 'love', 'dtf', 'not', 'this', 'location', 'is', 'there', 'anything', 'good', 'we', 'can', 'say', 'about', 'this', 'location', 'no', 'from', 'the', 'moment', 'we', 'walked', 'up', 'to', 'put', 'our', 'party', 'of', 'two', 'name', 'in', 'at', 'on', 'monday', 'afternoon', 'it', 'was', 'awful', 'the', 'two', 'receptionists', 'talked', 'in', 'whispers', 'behind', 'their', 'masks', 'and', 'when', 'we', 'asked', 'them', 'to', 'repeat', 'what', 'they', 'were', 'saying', 'the', 'spoke', 'at', 'mph', 'shouting', 'we', 'are', 'not', 'deaf', 'nor', 'stupid', 'and', 'we', 'should', 'have', 'walked', 'away', 'right', 'then', 'we', 'did', 'not', 'our', 'order', 'was', 'simple', 'veggie', 'two', 'bun', 'orders', 'appetizer', 'and', 'noodles', 'plus', 'hot', 'tea', 'the', 'meal', 'took', 'total', 'of', 'minutes', 'to', 'be', 'served', 'as', 'it', 'dripped', 'and', 'drabbled', 'out', 'of', 'the', 'kitchen', 'the', 'food', 'was', 'cold', 'tough', 'unappetizing', 'the', 'wait', 'staff', 'was', 'sn

In [32]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words) #, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['we', 'love', 'dtf', 'not', 'this_location', 'is', 'there', 'anything', 'good', 'we', 'can_say', 'about', 'this_location', 'no', 'from', 'the', 'moment', 'we', 'walked', 'up', 'to', 'put', 'our', 'party', 'of', 'two', 'name', 'in', 'at', 'on_monday', 'afternoon', 'it', 'was', 'awful', 'the', 'two', 'receptionists', 'talked', 'in', 'whispers', 'behind', 'their', 'masks', 'and', 'when', 'we', 'asked', 'them', 'to', 'repeat', 'what', 'they', 'were', 'saying', 'the', 'spoke', 'at', 'mph', 'shouting', 'we', 'are', 'not', 'deaf', 'nor', 'stupid', 'and', 'we', 'should_have', 'walked_away', 'right', 'then', 'we', 'did_not', 'our', 'order', 'was', 'simple', 'veggie', 'two', 'bun', 'orders', 'appetizer', 'and', 'noodles', 'plus', 'hot', 'tea', 'the', 'meal', 'took', 'total', 'of', 'minutes', 'to', 'be', 'served', 'as', 'it', 'dripped', 'and', 'drabbled', 'out', 'of', 'the', 'kitchen', 'the', 'food', 'was', 'cold', 'tough', 'unappetizing', 'the', 'wait', 'staff', 'was', 'snotty', 'arrogant', 'an

In [33]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [34]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ','ADV'])

print(data_lemmatized[:1])

[['love', 'location', 'good', 'location', 'moment', 'party', 'name', 'afternoon', 'receptionist', 'whisper', 'mask', 'repeat', 'mph', 'deaf', 'stupid', 'walked_away', 'right', 'order', 'simple', 'veggie', 'bun', 'order', 'appetizer', 'noodle', 'hot', 'tea', 'meal', 'total', 'minute', 'kitchen', 'cold', 'tough', 'unappetizing', 'staff', 'snotty', 'arrogant', 'noisy', 'floor', 'open_table', 'everywhere', 'server', 'kitchen', 'problem', 'today', 'rush', 'segment', 'tea', 'bill', 'dollar', 'short', 'afternoon', 'night', 'long', 'travel', 'lot', 'lot', 'place', 'world', 'stomach', 'type', 'mean', 'undercooked', 'never', 'ever', 'awful', 'place', 'business', 'month', 'figure', 'problem', 'right', 'retail', 'year', 'store', 'experience', 'authority', 'corporate', 'need', 'right', 'south_coast', 'plaza', 'retail', 'space', 'expensive', 'quickly', 'embarrassed', 'shocked', 'well', 'other', 'store', 'corporate', 'kind', 'problem', 'location', 'far', 'large', 'dissatisfied', 'former', 'customer']

In [73]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 3), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 2), (50, 1), (51, 3), (52, 1), (53, 1), (54, 1), (55, 2), (56, 3), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1)]]


In [113]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           passes=50,
                                           alpha='auto',
                                           per_word_topics=True)

In [114]:
# Print the Keyword in the 3 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.015*"table" + 0.015*"time" + 0.013*"hour" + 0.012*"reservation" + '
  '0.010*"customer" + 0.009*"people" + 0.008*"order" + 0.008*"party" + '
  '0.007*"minute" + 0.007*"service"'),
 (1,
  '0.019*"place" + 0.018*"service" + 0.018*"table" + 0.016*"good" + '
  '0.013*"server" + 0.013*"time" + 0.012*"order" + 0.011*"bad" + 0.010*"even" '
  '+ 0.009*"well"'),
 (2,
  '0.017*"good" + 0.014*"place" + 0.011*"dish" + 0.009*"chicken" + '
  '0.008*"rice" + 0.008*"well" + 0.007*"really" + 0.007*"pork" + '
  '0.007*"flavor" + 0.006*"great"')]


In [115]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.2543370538546075

Coherence Score:  0.47551566684335134


In [116]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = plt_gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.019789 -0.105162       1        1  69.985238
2     -0.150434  0.062316       2        1  16.215068
0      0.170223  0.042846       3        1  13.799694, topic_info=            Term         Freq        Total Category  logprob  loglift
148        table  1193.000000  1193.000000  Default  30.0000  30.0000
100         hour   440.000000   440.000000  Default  29.0000  29.0000
182         time   971.000000   971.000000  Default  28.0000  28.0000
177  reservation   198.000000   198.000000  Default  27.0000  27.0000
10      customer   333.000000   333.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
192         rude    41.319613   204.760963   Topic3  -5.6093   0.3800
23          good    61.159693  1220.546021   Topic3  -5.2171  -1.0130
408          bar    38.685531   178.013386   Topic3  -5.6752   0.4541
120        great    43.375308   407.161029   Topic3  -5.5607  -0.2588
49         place    44.544238  1325.815603   Topic3  -5.5341  -1.4128

[246 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
323       3  0.921350    advance
5337      2  0.941904        ahi
3206      1  0.989815      aisle
5888      3  0.944431      album
3020      3  0.928416    allergy
...     ...       ...        ...
80        2  0.165249       well
80        3  0.042092       well
1913      1  0.994831      young
1800      2  0.936773      yummy
467       3  0.927949  zero_star

[348 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

# Positive sentiment topic

In [48]:
# Select 4 and 5 star reviews as positive

pos_sent = reviews1.loc[(reviews1.stars==5) | (reviews1.stars==4)]
pos_sent

,username,location,date,stars,content,numfriends,restaurantname
0,Sarah E.,"Arcadia, CA",4/2/22,5,Been going here for 5+ years starting at the o...,247,Din Tai Fung
1,Cynthia W.,"Irvine, CA",3/17/22,4,Very professional and organized. Wonderful sta...,0,Din Tai Fung
2,Julienne C.,"Buena Park, CA",4/24/22,5,We came here for our anniversary as walk ins a...,0,Din Tai Fung
3,Tina N.,"Atlanta, GA",5/1/22,4,DTF? Obviously had to go to DTF for the one ti...,2,Din Tai Fung
5,Gerry T.,"Skokie, IL",4/23/22,5,"Absolutely love this place, great location too...",1528,Din Tai Fung
...,...,...,...,...,...,...,...
20975,nikki c.,"Oakland, CA",8/13/11,5,"This place is amazing. The ambiance is great,...",13,Habana
20976,Sal R.,"McLean, VA",4/12/12,5,One of my favorite restaurants in OC. I love t...,0,Habana
20977,Pranay N.,"Sunnyvale, CA",10/8/13,5,"they have the amazing food, the flavors are so...",64,Habana
20982,Thenameisfood T.,"Hacienda Heights, CA",9/11/12,4,pretty good food! bit pricey.. the Cuban Food ...,514,Habana


In [49]:
df2 = pos_sent[['content','stars']]
df2

,content,stars
0,Been going here for 5+ years starting at the o...,5
1,Very professional and organized. Wonderful sta...,4
2,We came here for our anniversary as walk ins a...,5
3,DTF? Obviously had to go to DTF for the one ti...,4
5,"Absolutely love this place, great location too...",5
...,...,...
20975,"This place is amazing. The ambiance is great,...",5
20976,One of my favorite restaurants in OC. I love t...,5
20977,"they have the amazing food, the flavors are so...",5
20982,pretty good food! bit pricey.. the Cuban Food ...,4


In [50]:
# Convert to list
data2 = df2.content.values.tolist()

data2 = [re.sub('\"', ' ', review) for review in data2]

# Remove new line characters
data2 = [re.sub('\s+', ' ', sent) for sent in data2]

# Remove distracting single quotes
data2 = [re.sub("\'", " ", sent) for sent in data2]

pprint(data2[:1])


['Been going here for 5+ years starting at the original in Arcadia. Amazing '
 'every single time. No complaints.']


In [51]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words2 = list(sent_to_words(data2))

print(data_words2[:1])

[['been', 'going', 'here', 'for', 'years', 'starting', 'at', 'the', 'original', 'in', 'arcadia', 'amazing', 'every', 'single', 'time', 'no', 'complaints']]


In [52]:
# Build the bigram and trigram models
bigram2 = gensim.models.Phrases(data_words2) #, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram2 = gensim.models.Phrases(bigram2[data_words2], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod2 = gensim.models.phrases.Phraser(bigram2)
trigram_mod2 = gensim.models.phrases.Phraser(trigram2)

# See trigram example
print(trigram_mod2[bigram_mod2[data_words2[0]]])

['been', 'going', 'here', 'for', 'years', 'starting', 'at', 'the', 'original', 'in', 'arcadia', 'amazing', 'every_single', 'time', 'no_complaints']


In [53]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod2[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod2[bigram_mod2[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [54]:
# Remove Stop Words
data_words_nostops2 = remove_stopwords(data_words2)

# Form Bigrams
data_words_bigrams2 = make_bigrams(data_words_nostops2)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized2 = lemmatization(data_words_bigrams2, allowed_postags=['NOUN', 'ADJ', 'ADV']) # 'VERB'

print(data_lemmatized2[:1])

[['year', 'original', 'arcadia', 'amazing', 'complaint']]


In [55]:
# Create Dictionary
id2word2 = corpora.Dictionary(data_lemmatized2)

# Create Corpus
texts2 = data_lemmatized2

# Term Document Frequency
corpus2 = [id2word2.doc2bow(text) for text in texts2]

# View
print(corpus2[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [122]:
# Build LDA model
lda_model2 = gensim.models.ldamodel.LdaModel(corpus=corpus2,
                                           id2word=id2word2,
                                           num_topics=3, 
                                           random_state=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [123]:
# Print the Keyword in the 3 topics
pprint(lda_model2.print_topics())
doc_lda2 = lda_model2[corpus2]

[(0,
  '0.034*"good" + 0.027*"place" + 0.026*"service" + 0.025*"great" + '
  '0.017*"meat" + 0.016*"time" + 0.012*"always" + 0.010*"really" + '
  '0.010*"server" + 0.009*"amazing"'),
 (1,
  '0.024*"good" + 0.019*"great" + 0.017*"place" + 0.011*"dish" + '
  '0.011*"really" + 0.010*"nice" + 0.009*"also" + 0.008*"friend" + '
  '0.008*"dinner" + 0.008*"drink"'),
 (2,
  '0.025*"good" + 0.023*"strawberry_croissant" + 0.022*"croissant" + '
  '0.021*"pastry" + 0.014*"place" + 0.014*"cream" + 0.013*"sweet" + '
  '0.012*"bread" + 0.011*"also" + 0.010*"strawberry"')]


In [124]:
# Compute Perplexity
print('\nPerplexity: ', lda_model2.log_perplexity(corpus2))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda2 = CoherenceModel(model=lda_model2, texts=data_lemmatized2, dictionary=id2word2, coherence='c_v')
coherence_lda2 = coherence_model_lda2.get_coherence()
print('\nCoherence Score: ', coherence_lda2)


Perplexity:  -7.028313772505907


2022-06-08 17:01:26,599 : ERROR : worker encountered unexpected exception
Traceback (most recent call last):
  File "/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/gensim/topic_coherence/text_analysis.py", line 561, in run
    self.input_q = input_q
  File "/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/gensim/topic_coherence/text_analysis.py", line 581, in _run
    n_docs = 0
  File "/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/gensim/topic_coherence/text_analysis.py", line 353, in partial_accumulate
    This method does not ensure the co-occurrence matrix is in lil format and does not
  File "/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/gensim/topic_coherence/text_analysis.py", line 296, in accumulate
    for doc_num, virtual_document in windows:
  File "/Users/ysabelgonzalezrico/opt/anaconda3/lib/python3.7/site-packages/gensim/topic_coherence/text_analysis.py", line 360, in analyze_text
  File "/


Coherence Score:  0.5376165884667158


In [125]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis2 = plt_gensim.prepare(lda_model2, corpus2, id2word2)
vis2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.036253 -0.141846       1        1  56.177117
2     -0.159937  0.043727       2        1  23.752468
1      0.123685  0.098119       3        1  20.070414, topic_info=                      Term         Freq        Total Category  logprob  \
248                service  6674.000000  6674.000000  Default  30.0000   
6756  strawberry_croissant  2491.000000  2491.000000  Default  29.0000   
6737             croissant  2363.000000  2363.000000  Default  28.0000   
5710                pastry  2211.000000  2211.000000  Default  27.0000   
304                   meat  4271.000000  4271.000000  Default  26.0000   
...                    ...          ...          ...      ...      ...   
486                   love   543.318089  2952.350260   Topic3  -5.1124   
148                 pretty   498.218095  2087.926775   Topic3  -5.1991   
0                  amazing   550.194446  3354.822395   Topic3  -5.0999   
360             definitely   519.089307  3205.100999   Topic3  -5.1580   
110                  sweet   438.949378  1977.421063   Topic3  -5.3257   

      loglift  
248   30.0000  
6756  29.0000  
6737  28.0000  
5710  27.0000  
304   26.0000  
...       ...  
486   -0.0867  
148    0.1730  
0     -0.2020  
360   -0.2145  
110    0.1008  

[229 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
128       1  0.476666    also
128       2  0.306540    also
128       3  0.216642    also
17        1  0.812911  always
17        2  0.172778  always
...     ...       ...     ...
222       2  0.223614    well
222       3  0.193341    well
125       1  0.790179   worth
125       2  0.201240   worth
125       3  0.008527   worth

[387 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])